# Convolutional Neural Networks and Fashion-MNIST with civis-compute

This example fits a convolutional neural network classifier to the [Fashion-MNIST dataset](https://github.com/zalandoresearch/fashion-mnist). This dataset consists of 60,000 images of different articles of clothing and accessories. See this TesorFlow [tutorial](https://www.tensorflow.org/get_started/mnist/pros) for an introduction to the underlying methodology.

In order to run the example locally, you need to have [muffnn](https://github.com/civisanalytics/muffnn) and [TensorFlow](https://www.tensorflow.org/) installed. This can be done with `pip`:

```
pip install muffnn tensorflow
```

To submit the example to Civis Platform, do the following:

```
$ civis-compute submit fashion_mnist.ipynb
<JOBID>
$ civis-compute status <JOBID>
$ civis-compute get <JOBID>
```

In [1]:
import os
import pickle
import tensorflow as tf
import numpy as np

from cnn import ConvNetClassifier

# These comments tell Civis Platform how to run the code.
#CIVIS name=fashion mnist
#CIVIS required_resources={cpu: 8192, memory: 16384, disk_space: 10.0}
#CIVIS docker_image_name=civisanalytics/datascience-python
#CIVIS docker_image_tag=3.2.0
#CIVIS files=cnn.py  # files here get uploaded with your script and placed at the same path

### Download the Train and Test Data

In [2]:
# The data is in the repo, so let's pull that down and read it in.
!rm -rf fashion-mnist && \
    git clone https://github.com/zalandoresearch/fashion-mnist.git && \
    cp fashion-mnist/utils/*.py . && \
    rm __init__.py

import mnist_reader
Xtrn, ytrn = mnist_reader.load_mnist('fashion-mnist/data/fashion', kind='train')
Xtst, ytst = mnist_reader.load_mnist('fashion-mnist/data/fashion', kind='t10k')

Xtrn = Xtrn.astype(np.float32)
ytrn = ytrn.astype(np.int32)
Xtst = Xtst.astype(np.float32)
ytst = ytst.astype(np.int32)

# Normalize the data to [0, 1].
Xtrn /= 255.0
Xtst /= 255.0

Cloning into 'fashion-mnist'...
remote: Counting objects: 391, done.
remote: Total 391 (delta 0), reused 0 (delta 0), pack-reused 391
Receiving objects: 100% (391/391), 103.01 MiB | 4.19 MiB/s, done.
Resolving deltas: 100% (211/211), done.


### Instantiate and Train the Classifier

We are adding a little dropout for regularization and turning down the learning rate accordingly.

In [3]:
cnn = ConvNetClassifier(solver_kwargs={'learning_rate': 1e-3}, keep_prob=0.8, n_epochs=1)

n_subepochs = 5
subepoch_size = Xtrn.shape[0] // n_subepochs

for i in range(5):
    for subepoch in range(n_subepochs):
        start = subepoch * subepoch_size
        end = min(start + subepoch_size, Xtrn.shape[0])
        cnn.partial_fit(Xtrn[start:end], ytrn[start:end], classes=np.unique(ytrn))

        print('epoch %0.2f: test accuracy = %0.3f' % (
            i + (subepoch + 1) / n_subepochs, cnn.score(Xtst, ytst)), flush=True)

epoch 0.20: test accuracy = 0.847
epoch 0.40: test accuracy = 0.868
epoch 0.60: test accuracy = 0.865
epoch 0.80: test accuracy = 0.874
epoch 1.00: test accuracy = 0.890
epoch 1.20: test accuracy = 0.888
epoch 1.40: test accuracy = 0.899
epoch 1.60: test accuracy = 0.892
epoch 1.80: test accuracy = 0.896
epoch 2.00: test accuracy = 0.902
epoch 2.20: test accuracy = 0.908
epoch 2.40: test accuracy = 0.908
epoch 2.60: test accuracy = 0.911
epoch 2.80: test accuracy = 0.911
epoch 3.00: test accuracy = 0.916
epoch 3.20: test accuracy = 0.912
epoch 3.40: test accuracy = 0.911
epoch 3.60: test accuracy = 0.912
epoch 3.80: test accuracy = 0.916
epoch 4.00: test accuracy = 0.917
epoch 4.20: test accuracy = 0.907
epoch 4.40: test accuracy = 0.912
epoch 4.60: test accuracy = 0.915
epoch 4.80: test accuracy = 0.915
epoch 5.00: test accuracy = 0.921


### Serialize the Classifier for Later

`muffnn` automatically handles serialization of the `TensorFlow` graph and session, so we just have to pickle the model and save it in the right spot. Then `civis-compute` takes care of pushing it to Civis Platform file storage.

In [4]:
# We only save it if running in Platform.
if 'CIVIS_JOB_DATA' in os.environ:
    with open(os.path.expandvars(os.path.join('${CIVIS_JOB_DATA}', 'cnn.pkl')), 'wb') as fp:
        pickle.dump(cnn, fp)